In [1]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
os.getcwd()

C:\Users\Kotani Lab\AppData\Local\Temp\ipykernel_7412\1802075856.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


'C:\\Users\\Kotani Lab\\Desktop\\ML_senior_project\\ML-Based-Adaptive-Cybersecurity-Incident-Detection\\Code_and_model\\Program'

In [69]:
df = pd.read_csv('kdd/train_test_folder/train_kdd/train.csv')
traindf = pd.read_csv('./kdd/dataset/mix_dataset/neptune.csv')

In [70]:
df

,Unnamed: 0,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,label
0,90597,0.0,2.536298e-08,9.924138e-08,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,normal
1,89016,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neptune
2,106202,0.0,1.420327e-07,5.498736e-06,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,normal
3,74603,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,neptune
4,104247,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103956,38990,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,neptune
103957,122689,0.0,1.565258e-07,3.334510e-06,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,normal
103958,126235,0.0,1.920340e-07,2.693258e-06,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,normal
103959,10237,0.0,7.478457e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,smurf


In [74]:
90597 - (window_size - 1)


90086

In [102]:
window_size = 512
batch_size = 2
epochs = 20
X = traindf.drop('label', axis=1)
y = traindf['label']

Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)

In [103]:
original_index = 90597

In [104]:
traindf.iloc[original_index]
array = traindf.drop('label', axis=1).iloc[original_index].to_numpy()

datagen = Data[90086][0][0][511]
np.array_equal(array, datagen)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [105]:
sequence_start_index = original_index - (window_size - 1)
batch_index = sequence_start_index // batch_size
position_within_batch = sequence_start_index % batch_size


In [106]:
sequence_start_index

90086

In [107]:
batch_index

45043

In [108]:
position_within_batch

0

In [113]:
Data[batch_index][0][0][511]

array([0.00000000e+00, 2.53629825e-08, 9.92413835e-08, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.91389432e-03,
       1.95694716e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.00000000e-01, 1.00000000e+00, 0.00000000e+00,
       7.80392157e-01, 3.13725490e-02, 4.00000000e-02, 4.00000000e-02,
       1.00000000e-02, 2.50000000e-01, 1.00000000e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
os.getcwd()

'C:\\Users\\Kotani Lab\\Desktop\\ML_senior_project\\ML-Based-Adaptive-Cybersecurity-Incident-Detection\\Code_and_model\\Program'

In [2]:
df = pd.read_csv('kdd/train_test_folder/train_kdd/train.csv')
traindf = pd.read_csv('./kdd/dataset/mix_dataset/neptune.csv')

In [3]:
from multiprocessing import Pool, cpu_count

window_size = 64
batch_size = 1
epochs = 20

X = traindf.drop('label', axis=1)
y = traindf['label']

Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)

def process_batch(args):
    print('Processing1....')
    batch_range, generator, df_index, batch_size, window_size, show_progress = args
    rearranged_batch_data = []
    print('Processing3....')
    # Wrap the enumeration with tqdm if this is the first subprocess to show progress
    iterable = enumerate(preloaded_data)
    if show_progress:
        iterable = tqdm(iterable, total=len(batch_range), desc="Processing batches")

    for idx, (batch_x, batch_y) in iterable:
        i = batch_range[idx]
        for j in range(batch_x.shape[0]):
            sequence_start_index = i * batch_size + j
            original_indices = df_index[sequence_start_index: sequence_start_index + window_size].tolist()
            rearranged_batch_data.append((batch_x[j], batch_y[j], original_indices))

    return rearranged_batch_data

def rearrange_sequences_multiprocessing(generator, df_index, batch_size, window_size):
    num_batches = len(generator)
    num_processes = cpu_count()

    # Split the batch indices into approximately equal chunks for each process
    batch_ranges = np.array_split(range(num_batches), num_processes)

    # Prepare the arguments for each process, setting show_progress=True only for the first batch
    print('Prepare the arguments...')
    process_args = [(batch_range, generator, df_index, batch_size, window_size, i == 0) 
                for i, batch_range in tqdm(enumerate(batch_ranges), total=len(batch_ranges), desc="Preparing process arguments")]

    print('Done...')
    rearranged_data = []
    with Pool(num_processes) as pool:
        results = pool.map(process_batch, process_args)

        # Combine the results from all processes
        for result in results:
            rearranged_data.extend(result)

    return rearranged_data

In [ ]:
rearranged_data = rearrange_sequences_multiprocessing(Data, df['Unnamed: 0'], batch_size, window_size)

Prepare the arguments...


Preparing process arguments: 100%|██████████| 12/12 [00:00<?, ?it/s]


Done...


In [118]:
def verify_sequence_order(rearranged_data, original_indices):
    for sequence, label, seq_original_indices in rearranged_data:
        expected_start_index = seq_original_indices[0]
        
        actual_start_pos = original_indices.index(expected_start_index)
        
        expected_indices = original_indices[actual_start_pos:actual_start_pos + len(seq_original_indices)]
        
        expected_set = set(expected_indices)
        actual_set = set(seq_original_indices)
        
        if expected_set != actual_set:
            print(f"Mismatch found: Expected {expected_set}, Got {actual_set}")
            return False
    
    print("All sequences match the original order of indices.")
    return True


original_indices = df['Unnamed: 0'].tolist()
verification_result = verify_sequence_order(rearranged_data, original_indices)

print(f"Verification Result: {verification_result}")


KeyboardInterrupt

